In [1]:
%cd ..

/home/pauline/Dokumente/Delab_Projekt/delab


In [2]:
import pandas as pd

In [3]:
from util.sql_switch import get_query_native

df = get_query_native(
    "SELECT author_id, conversation_id, name , is_climate_author, climate_author_type FROM delab_tweet, delab_tweetauthor WHERE delab_tweet.author_id=delab_tweetauthor.twitter_id")
df2 = df.loc[df['is_climate_author']== True]
df3 = get_query_native("SELECT * FROM delab_conversation")
df4 = pd.merge(df2,df3, on='conversation_id')
df4.head(100)

using postgres
using postgres


/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/pauline/Dokumente/Delab_Projekt/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,author_id,conversation_id,name,is_climate_author,climate_author_type,id,depth,branching_factor,root_dominance
0,3459051,1578218562170155009,Greenpeace,True,non-governmental organisation,8,3,5.0,0.166667
1,17463923,1577945023815065602,UN Climate Change,True,organisation,6,3,51.0,0.019231
2,3459051,1577040800994983936,Greenpeace,True,non-governmental organisation,4,3,14.0,0.066667
3,384923143,1577469134119784448,Prof Michael E. Mann,True,scientist,5,2,6.0,0.142857
4,1096766566279073792,1585616351493820416,ParentsForFuture #ExitECT #EndFossilFuels,True,organisation,19,3,21.0,0.642857
5,1008688766062661632,1585616351493820416,Nyombi Morris,True,activist,19,3,21.0,0.642857
6,1008688766062661632,1585616351493820416,Nyombi Morris,True,activist,19,3,21.0,0.642857
7,1008688766062661632,1585616351493820416,Nyombi Morris,True,activist,19,3,21.0,0.642857
8,1008688766062661632,1585616351493820416,Nyombi Morris,True,activist,19,3,21.0,0.642857
9,1008688766062661632,1585616351493820416,Nyombi Morris,True,activist,19,3,21.0,0.642857
